In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities= cities.iloc[:-1,[0,3,5,6,7,8]]

    #We will now perform a basic data clean-up process that we came up with on our last data set

    def clean_up_cities_rules(cities, sport):     #Let's make our data easier to manage 
        cities.rename(columns = {"Population (2016 est.)[8]":"Population", "Metropolitan area":"City"}, inplace = True)
        #We keep only the relevant to the sport team
        cities = cities[["City","Population",sport]]
        #We replace the -- with NaN cause it's easier to handle
        cities.loc[:,sport] = cities.loc[:,sport].replace("—",np.nan)
        cities = cities.dropna()
        #We want to replace everything in [] with ""
        cities.loc[:,sport] = cities.loc[:,sport].replace("\[.*\]","",regex = True)
        #Anything that is empty at the sports column will be the ones that don't have this sport on the city
        cities.loc[:,sport] = cities.loc[:,sport].replace("",np.nan)
        cities = cities.dropna()
        return cities

    metro_rules_MLB = clean_up_cities_rules(cities,"MLB")

    #Now let's do a similar clean-up process for the NBA sheet
    #We will do it by hand first and maybe we'll write a function later 
    mlb_stats = pd.read_csv("assets/mlb.csv")

    #Keeping only the 2018 data 
    mask2018 = mlb_stats["year"] == 2018
    mlb_stats = mlb_stats[mask2018]

    #Okay now we have the teams of 2018 MLB
    #From this table we only want the teams and their W/L
    mlb_stats = mlb_stats.loc[:,["team","W-L%"]]
    #We also want to make our city column
    mlb_stats["City"] = np.nan

    #Let's do the same procedure
    #This is the function that will find the problematic team names 
    easy_named_teams = []
    problematic_names = []
    #This function can check if the city of the team in the nhl_stats is found in 
    #our cities column
            
            
    def assign_city(tname): 
        if tname.split()[0] in str(metro_rules_MLB["City"].unique()):
            easy_named_teams.append(tname)
        else: 
            problematic_names.append(tname)

    mlb_stats["team"].apply(lambda x: assign_city(x)) #This will give us the nice lists we want
    
    #Let's actually do the easy ones first
    #Beautiful let's now fix up our other teams as well. 
    def find_city_by_team(tname):
        result = metro_rules_MLB[metro_rules_MLB["City"].str.startswith(tname.split()[0])]
        #Get the City columns that start with the first word of my tname
        return result["City"].iloc[0]
    
    def team_rename(row, dictionary): 
        team_name = row["team"] #Get the team name from the team column 
        if team_name in dictionary.keys(): #If it's in that mapping add the value in the City col
            row["City"] = dictionary[team_name]
        return row
    
    easy_named_teams_dict = {}
    
    for team in easy_named_teams: 
        try: easy_named_teams_dict[team] = find_city_by_team(team)
        except: continue

    mlb_stats = mlb_stats.apply(team_rename, axis = 1, dictionary = easy_named_teams_dict)
    #Let's now do the problematic ones 
    #Let's not forget to add the other ones I spotted by eye[San Diego one]
    problematic_mapping = {
        'Minnesota Twins': 'Minneapolis–Saint Paul',
        'Oakland Athletics': 'San Francisco Bay Area',
        'Texas Rangers': 'Dallas–Fort Worth',
        'Colorado Rockies': "Denver",
        'Arizona Diamondbacks': "Phoenix",
       'San Diego Padres': 'San Diego'
    }

    #We will now apply this with the dictionary = problematic_mapping 
    mlb_stats = mlb_stats.apply(team_rename, axis = 1, dictionary = problematic_mapping)

    #Now that we're good City name wise let's gather up them means 
    mlb_stats["W-L%"] = mlb_stats["W-L%"].astype(float)
    mean_wl = mlb_stats["W-L%"].groupby(mlb_stats["City"]).mean()
    final = pd.merge(metro_rules_MLB, mean_wl, how = "inner", on = "City")
    correlation = stats.pearsonr(final["Population"].astype(float), final["W-L%"].astype(float))

    return correlation[0]